# SAP SSFS

The following subsections show a representation of the file format portions and how to generate them.

First we need to perform some setup to import the packet classes:

In [ ]:
from pysap.SAPSSFS import *
from pysap.utils.crypto import rsec_decrypt
from IPython.display import display

## SSFS files

We'll read the key and data files used in the test case suite and use them as example:

In [ ]:
with open("../../tests/data/ssfs_hdb_dat", "rb") as fd:
    data = fd.read()
    
ssfs_data = SAPSSFSData(data)

with open("../../tests/data/ssfs_hdb_key", "rb") as fd:
    key = fd.read()

ssfs_key = SAPSSFSKey(key)

SSFS files are comprised of the following main structures:

### SSFS Data

In [ ]:
ssfs_data.show()

As can be observed, a SSFS Data file contains multiple records with different key/value pairs, as well as associated meta data.

Some records contain values stored in plaintext, while others are stored in an encrypted fashion. We'll see a password record, which is stored encrypted:

In [ ]:
ssfs_data.records[-1].canvas_dump()

Additionally, each SSFS record contains an HMAC-SHA1 value calculated using a fixed key. The intent of this value is to provide integrity validation as well as ensure that an authentic tool was used to generate the files:

In [ ]:
ssfs_data.records[-1].valid

### SSFS Key content

In [ ]:
ssfs_key.canvas_dump()

## SSFS Value access

The values contained in SSFS Data records can be accessed by providing the key name:

In [ ]:
ssfs_data.get_value('HDB/KEYNAME/DB_USER')

### SSFS Data content decryption

For those records that are stored encrypted, it's possible to access the right value by providing the key name and the proper SSFS decryption key structure:

In [ ]:
ssfs_data.get_value('HDB/KEYNAME/DB_PASSWORD', ssfs_key)

## SSFS Decrypted Payload structure

The decryption mechanism can be user to obtain the raw data stored encrypted:

In [ ]:
decrypted_blob = rsec_decrypt(ssfs_data.get_record('HDB/KEYNAME/DB_PASSWORD').data, ssfs_key.key)
decrypted_blob

It's possible also to parse that raw data and obtain the underlying strucutures and meta data associated:

In [ ]:
payload = SAPSSFSDecryptedPayload(decrypted_blob)
payload.canvas_dump()

The decrypted payload contains a hash calculated using the SHA-1 algorithm, and that can be used to validate integrity of the entire payload:

In [ ]:
payload.valid